In [1]:
# Импортирование библиотек для иницализации основных параметров системы
from fortran_calc_tools import *

# Импортирование неодходимых библиотек
from os import path 
import time
import pandas as pd, numpy as np
from collections import namedtuple
from calc_tools import pairwise_1, pairwise_2
from scipy.linalg import logm, expm, sqrtm, eig, lu, lu_factor, lu_solve
from multiprocessing import Pool
from numpy.linalg import inv
from numpy import linalg as LA

# Импортирование библиотек для визуализации данных !!!!!
import matplotlib.pyplot as plot
import matplotlib as mpl
import seaborn as sns
import plotly as pt
import plotly.express as px
%matplotlib inline
import plotly.graph_objects as go

In [2]:
# Задание значении характеристик материала для фаз и троса
#                         GM    OPM    S     XA   YA
Faza_A = Fazy("Фаза А", 35.336, 1.0, 150.0, 0.0, 19.0)
Faza_B = Fazy("Фаза В", 35.336, 1.0, 150.0, 6.3, 19.0)
Faza_C = Fazy("Фаза С", 35.336, 1.0, 150.0, 4.2, 25.0)
Tross = Fazy("Тросс", 17.336, 4000.0, 50.0, 2.1, 28.0)

# В данном случае у нас будет 50 гармоник от [1 до 50]
garmoniki = Garmoniki(50)

# Создание объекта для проведения расчетов
rashet = Rachety()

# Инициализация объекта для подстанции "Зеленая" и чтение файла
basepath = path.abspath('')
filepath = path.abspath(path.join(basepath, "..", path.join("data", "Данные ПС Сарепта1.xls")))
PodS_zelenaya = PodStans(filepath, volt=110, harm_num=49)

dummy_str_input = "fuzzBuzz for testing"
dummy_int_input = 404

system = System(PodS_zelenaya, garmoniki, rashet, Fazy, dummy_str_input, dummy_str_input, dummy_int_input, dummy_int_input,
     dummy_int_input, dummy_int_input, dummy_int_input)

# Установка конфигурации для выведения данных
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

In [3]:
#                   !!! DEBUGGING PROCESS HERE !!!

In [11]:
# GETTING MAIN LINE CHARACTERISTICS FROM EXCEL FILE
UM, FUM, AIM, FIM, funu, fu, funi, fi, knsu, knsi, rmsu, rmsi = system.line_matrix(1)

In [12]:
# DECLARING VARIABLES

WD = np.zeros((2, 50))
PD = np.zeros((3, 700))

UM1 = np.zeros((700, UM.shape[1],UM.shape[2]))
UM2 = np.zeros((700, UM.shape[1],UM.shape[2]))
AIM1 = np.zeros((700, UM.shape[1],UM.shape[2]))
AIM2 = np.zeros((700, UM.shape[1],UM.shape[2]))
PPR1, PPR2 = np.zeros((700)), np.zeros((700))

PPP = np.zeros((1000, 50)); PPP1 = np.zeros((1000, 50));
PPP2 = np.zeros((1000, 50)); PPP3 = np.zeros((1000, 50));
PPP4 = np.zeros((1000, 50)); PPP5 = np.zeros((1000, 50));
PPP6 = np.zeros((1000, 50)); PPP7 = np.zeros((1000, 50));
PPP8 = np.zeros((1000, 50));

# !!!!!!!!!!!!!!!!! CordN = 8 was in original here 
cordN = 4
UK1 = np.array([complex(0, 0) for _ in range(cordN)], dtype="complex128")
AIK1 = np.array([complex(0, 0) for _ in range(cordN)], dtype="complex128")

UK10 = complex(0, 0); UK11 = complex(0, 0); UK12 = complex(0, 0);
AIK10 = complex(0, 0); AIK11 = complex(0, 0); AIK12 = complex(0, 0);
AL = complex(0, 0);

PP1 = 0.0
PP2 = 0.0
RZ = 35.3

In [13]:
# Implementing RASCHET fortran's subroutine in python
def RASCHET(NN, LL):
# LL is in range [0-559] NN [0-49]

#     print(NN, LL)
    global UK1, AIK1, PPP,PP1,PP2,PPP1,PPP2,PPP3,PPP4,PPP5,PPP6,PPP7,PPP8, MM,M,M1,MT,M10,M20,PR,K1,K2,K3,N1,N2,N3,MPR,MTR,MMT
    
    
    XA, YA, OMP, GM, S = Fazy.construc_matrices()
    R0, R, UXM, HI, R11, DET2, DET4 = np.zeros((M)), np.zeros((M)), np.zeros((M)), np.zeros((M)), np.zeros((M)), np.zeros((M)), np.zeros((M))
    EVU, B, UX, AIX, SM, EVI, DET1, DET3, BB, AIXM, AA = np.zeros((M)), np.zeros((M), dtype="complex128"), np.zeros((M), dtype="complex128"), np.zeros((M), dtype="complex128"), np.zeros((M), dtype="complex128"), np.zeros((M)), np.zeros((M), dtype="complex128"), np.zeros((M), dtype="complex128"), np.zeros((M)), np.zeros((M)), np.zeros((M)),
    B1, B4, B5 = np.zeros((M20), dtype="complex128"), np.zeros((M20)), np.zeros((M20), dtype="complex128")
    B6, B10, B7 = np.zeros((M10)), np.zeros((M10), dtype="complex128"), np.zeros((M10)) 
    
    DET10, DET20, SS, SS1, EX1,  = [], [], [], [], []
    
    HC1, HC2, HC3, HC4, F10, XL, XL1, G, D, HC, UXM = np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M)) 
    Z, Y, AU, E,  AAI, F, F1 = np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128"), np.zeros((M, M)), np.zeros((M, M), dtype="complex128"), np.zeros((M, M)), np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128")
    F2, D1, D2, D3, LU, LI = np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128")
    LU1, LI1, LU2, LU3, LI2, LI3 = np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128")
    F3, F4, F5, F6, F7 = np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M))
    HH13, HH14, HH21, HH22, HH23 = np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128")
    HH31, HH32, HH33, HH34, HH24 = np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128")
    HH41, HH42, HH43, HH44, HH = np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128"), np.zeros((M10, M10))
    CC, DD, HH11, HH12 = np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128")
    A1, A2 = np.zeros((M1, M1), dtype="complex128"), np.zeros((M1, M1), dtype="complex128")
    
    AG, GG = np.zeros((M1, M, M), dtype=type), np.zeros((M20, M20), dtype="complex128")
    GG1, GG2, GG3, GG4, GG5 = np.zeros((M20, M20), dtype="complex128"), np.zeros((M20, M20), dtype="complex128"), np.zeros((M10, M20), dtype="complex128"), np.zeros((M10, M10), dtype="complex128"), np.zeros((M10, M10), dtype="complex128")
    IPVT1, IH = np.zeros((M1)), np.array([1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0])
    
    if PR == 1:
        PP1 = 0
    if PR == 2:
        PP2 = 0
#     PPP[NN, LL] = 0
#     PPP1[NN, LL] = 0
#     PPP2[NN, LL] = 0
#     PPP3[NN, LL] = 0
#     PPP4[NN, LL] = 0
#     PPP5[NN, LL] = 0
#     PPP6[NN, LL] = 0
#     PPP7[NN, LL] = 0
#     PPP8[NN, LL] = 0
    
    MMT = MM / MT
    W = float(LL+1)
    EX1 = complex(2.71828, 0)
    
    print("AIK1: ", AIK1) # LEFT HERE !!!!
    
    # WRITING INTO UNIT 5. LINE MAIN CHARACTERISTIC
    
    for i in range(M):
        R[i]=np.sqrt(S[i]/np.pi)/1000.
        HI[i]=R[i]/(2.)*np.sqrt(2*np.pi*W*50*4*np.pi*OMP[i]*GM[i]/20)
        R0[i]=1000/(GM[i]*S[i])
        if HI[i] < 1:
            R11[i]=R0[i]*(1+HI[i]**4/3)
        if HI[i] > 1: 
            R11[i]=R0[i]*(HI[i]+0.25+3./(64.*HI[i]))
        if i+1 == M:
            pass
            # WRITE R11's values INTO THE FILE "R11"

#     print("W: ", W)
    
    # 845 - 12
    for i in range(M10):
        for j in range(M10):
            HH[i,j] = 0 

    # 12 - 161
    for i in range(M):
        for j in range(M):
            if i == j:
                D[i,i] = R[i]
            if i != j:
                D[i,j]=np.sqrt((XA[i]-XA[j])**2+(YA[i]-YA[j])**2)
            HC[i,j]=np.sqrt((XA[i]-XA[j])**2+(YA[i]+YA[j])**2)
            E[i,j]=complex(0.0, 0.0)
            E[i,i]=complex(1.0, 0.0)

    # 161 - 740
    for i in range(M):
        for j in range(M):
            XL1[i,j]=(0.145*np.log10(1000/D[i,j]))/(100*np.pi)
    
    # 740 - 743
    for i in range(M):
        for j in range(M):
            HC1[i,j]=41.4*10**6*np.log10(HC[i,j]/D[i,j])
    
    # 743 - 744
    HC1 = HC1.astype('float64')
    HC3 = DLINRG(HC1)
    F10 = DMRRRR(HC1,HC3)
    
    for i in range(M):
        for j in range(M):
            HC2[i,j]=HC3[i,j]*2.*np.pi*50
    
    # 744 - 847
    for i in range(M):
        for j in range(M):
            XL[i,j]=XL1[i,j]*W*2*50*np.pi
            HC4[i,j]=HC2[i,j]*W
            R10=0.0
            if i == j:
                Z[i,j]=complex(R11[i],XL[i,j]) 
            if i != j:
                Z[i,j]=complex(R10,XL[i,j])
            if i == j:
                G[i,j]=0.00000004*YA[i]/YA[i] 
            if i != j:
                G[i,j]=-0.00000004*YA[0]/D[i,j] 
            G[i,j]=0.
            Y[i,j]=complex(G[i,j],HC4[i,j])
    
    
    # 847 - the end of subroutine        
    for III in range(MT): # <----- main loop # 1300
#         print("Main loop counter MT: ", III)
        
        if M != 3: # GOTO 767
#             print("Here 3")
            if (M != 4): # GOTO 768
#                 print("Here 4")
                if (M != 6): # GOTO 769
#                     print("Here 6")
                    if (M != 7): # GOTO 770
#                         print("Here 7")
                        if (M != 8): # GOTO 771
                            pass
#                             print("Here 8")

                        else:
                            print("else 8")
                            # 770 - 766
                            for i in range(3):
                                B5[i]=UK1[i]
                                B5[i+3]=UK1[i]
                                B5[MPR+1-1]=complex(0.,0.)
                                B5[MPR+2-1]=complex(0.,0.)
                                B5[i+M]=AIK1[i]
                                B5[i+M+3]=AIK1[i]
                                B5[2*M-1-1]=complex(0.,0.)
                                B5[2*M-1]=complex(0.,0.)
                                B5[i+2*M]=complex(0.,0.)
                                B5[i+2*M+3]=complex(0.,0.)
                                B5[3*M-1-1]=complex(0.,0.)
                                B5[3*M-1]=complex(0.,0.)
                                B5[i+3*M]=complex(0.,0.)
                                B5[i+3*M+3]=complex(0.,0.)
                                B5[4*M-1-1]=complex(0.,0.)
                                B5[4*M-1]=complex(0.,0.)
                          
                    else:
                        print("else 7")
                        # 769 - 765
                        for i in range(3):
                            B5[i]=UK1[i]
                            B5[i+3]=UK1[i] 
                            B5[M-1]=complex(0.,0.) 
                            B5[i+M]=AIK1[i]
                            B5[i+M+3]=AIK1[i] 
                            B5[2*M-1]=complex(0.,0.)
                            B5[i+2*M]=complex(0.,0.)
                            B5[i+2*M+3]=complex(0.,0.)
                            B5[3*M-1]=complex(0.,0.)
                            B5[i+3*M]=complex(0.,0.)
                            B5[i+3*M+3]=complex(0.,0.)
                            B5[4*M-1]=complex(0.,0.)
                      

                else:
                    print("else 6")
                    # 768 - 764
                    for i in range(3):
                        B5[i]=UK1[i]
                        B5[i+3]=UK1[i]
                        B5[i+M]=AIK1[i]
                        B5[i+M+3]=AIK1[i]
                        B5[i+2*M]=complex(0.,0.)
                        B5[i+2*M+3]=complex(0.,0.)
                        B5[i+3*M]=complex(0.,0.)
                        B5[i+3*M+3]=complex(0.,0.)

            else:
#                 print("else 4")
                # 767 - 762
                for i in range(3):
    #   !!!!!!!!!!!!!!!!!!!!   AIK1 and UK1 ARE THE REAL EVILS IN THIS PARTY !!!!!!!!!!!!!!!!!!!!!!!!
#                     print("AIK1[i]: ", AIK1[i])
#                     print("UK1[i]: ",UK1[i])
                    B5[i]=UK1[i]
                    B5[M-1]=complex(0.,0.)
                    B5[i+M]=AIK1[i]
                    B5[2*M-1]=complex(0.,0.)
                    B5[i+2*M]=complex(0.,0.)
                    B5[3*M-1]=complex(0.,0.)
                    B5[i+3*M]=complex(0.,0.)
                    B5[4*M-1]=complex(0.,0.)
                
        else:
            print("else 3")
            # 847 - 761
            for i in range(3):
                B5[i]=UK1[i]
                B5[i+3]=AIK1[i]
                B5[i+6]=complex(0.,0.)
                B5[i+9]=complex(0.,0.)
            
# &&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&& DEBUGGING &&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
         
       #   !!!!!!!!!!!!!!!!!!!!   B5 IS THE REAL EVIL IN THIS PARTY !!!!!!!!!!!!!!!!!!!!!!!!
#         print("B5: ", B5)
        
        AU = DMCRCR(Z,Y)
        SS1=np.sqrt(AU[0,0])
        EVU = DEVLCG(AU)
        
        # IMPORTANT POINT
        for j in range(M):
            for i in range(M):
                F[i,j]=EVU[i]**j # (j-1) was in the first case
                F1[i,j]=F[i,j]
        
        for i in range(M1):
            for j in range(M1):
                A1[i,j]=F[i,j]

        A2 = DLFTCG(A1)
        SS = DET10 = DLFDCG(A2); DET20 = 1 # parameters DET10,DET20 were in function call
        # SS=DET10*(10.**DET20)
        
        for j in range(M1):

            for ii in range(M1):
                for jj in range(M1):
                    F[ii,jj]=F1[ii,jj]

            for i in range(M1):
                F[i,j]=EVU[i]**0.5

            for ii in range(M1):
                for jj in range(M1):
                    A1[ii,jj]=F[ii,jj]

            A2 = DLFTCG(A1)
            DET1[j] = DLFDCG(A2); DET2[j] = 1 # parameters DET1(J), DET2(J) were in function call

        F2 = DMCRCR(AU,AU)
        F3 = DMCRCR(F2,AU)
        F4 = DMCRCR(F3,AU)
        F5 = DMCRCR(F4,AU)
        F6 = DMCRCR(F5,AU)
        F7 = DMCRCR(F6,AU)
        
        for i in range(M1):
            for ii in range(M):
                for jj in range(M):

                    if i == 0:
                        AG[0,ii,jj]=E[ii,jj]
                    if i == 1:
                        AG[1,ii,jj]=AU[ii,jj]
                    if i == 2:
                        AG[2,ii,jj]=F2[ii,jj]
                    if i == 3:
                        AG[3,ii,jj]=F3[ii,jj]
                    if i == 4:
                        AG[4,ii,jj]=F4[ii,jj]
                    if i == 5:
                        AG[5,ii,jj]=F5[ii,jj]
                    if i == 6:
                        AG[6,ii,jj]=F6[ii,jj]
                    if i == 7:
                        AG[7,ii,jj]=F7[ii,jj]

        for ii in range(M):
            for jj in range(M):
                LU[ii,jj]=0.

        for i in range(M1):
            DET3[i]=DET1[i]/DET10
            DET4[i]=DET2[i]-DET20
            for ii in range(M):
                for jj in range(M):
                    LU[ii,jj]=LU[ii,jj]+AG[i,ii,jj]*DET3[i]*(10**DET4[i])
        
        F3 = DMCRCR(LU,LU)
        AAI = DMCRCR(Y,Z)
        EVI = DEVLCG(AAI)
  
        for j in range(M):
            for i in range(M):
                F[i,j] = EVI[i]**j # j-1
                F1[i,j] = F[i,j]

        for i in range(M1):
            for j in range(M1):
                A1[i,j]=F[i,j]

        A2 = DLFTCG(A1)
        DET10 = DLFDCG(A2); DET20 = 1 # parameters DET10,DET20 were in function call
        
        for j in range(M1):
            for ii in range(M1):
                for jj in range(M1):
                    F[ii,jj]=F1[ii,jj]

            for i in range(M1):
                F[i,j]=EVI[i]**0.5
            for ii in range(M1):
                for jj in range(M1):
                    A1[ii,jj]=F[ii,jj]

            A2 = DLFTCG(A1)
            DET1[j] = DLFDCG(A2); DET2[j] = 1 # parameters DET1(J),DET2(J) were in function call
            
        F2 = DMCRCR(AAI,AAI)
        F3 = DMCRCR(F2,AAI,)
        F4 = DMCRCR(F3,AAI)
        F5 = DMCRCR(F4,AAI)
        F6 = DMCRCR(F5,AAI)
        F7 = DMCRCR(F6,AAI)

        for i in range(M1):
            for ii in range(M):
                for jj in range(M):

                    if i == 0:
                        AG[0,ii,jj]=E[ii,jj]
                    if i == 1:
                        AG[1,ii,jj]=AAI[ii,jj]
                    if i == 2:
                        AG[2,ii,jj]=F2[ii,jj]
                    if i == 3:
                        AG[3,ii,jj]=F3[ii,jj]
                    if i == 4:
                        AG[4,ii,jj]=F4[ii,jj]
                    if i == 5:
                        AG[5,ii,jj]=F5[ii,jj]
                    if i == 6:
                        AG[6,ii,jj]=F6[ii,jj]
                    if i == 7:
                        AG[7,ii,jj]=F7[ii,jj]

        for ii in range(M):
            for jj in range(M):
                LI[ii,jj]=0.

        for i in range(M1):
            for ii in range(M):
                for jj in range(M):
                    LI[ii,jj]=LI[ii,jj]+(DET1[i]*(10**DET2[i])*AG[i,ii,jj])/(DET10*(10**DET20))
        
        F3 = DMCRCR(LI,LI)
        LM=MMT
        
        for n, _ in enumerate(range(2), start=1):

            if n == 1:
                SA=-1.
            if n == 2:
                SA=1.

            for ii in range(M):
                for jj in range(M):
                    LU1[ii,jj]=SA*MMT*LU[ii,jj]


            EVU = DEVLCG(LU1)
            for i in range(M):
                for j in range(M):
                    F[i,j]=EVU[i]**j # (j-1) here was an expression (j-1)


            for ii in range(M):
                for jj in range(M):
                    F1[ii,jj]=F[ii,jj]


            for ii in range(M1):
                for jj in range(M1):
                    A1[ii,jj]=F[ii,jj]
            
            A2 = DLFTCG(A1)
            DET10 = DLFDCG(A2); DET20 = 1 # DET10 and DET20 were the arguments of this function
            
            for j in range(M):
                for ii in range(M):
                    for jj in range(M):
                        F[ii,jj]=F1[ii,jj]


                for i in range(M):
#                     F = F.astype("complex128")
                    F[i,j]=EX1**EVU[i]


                for ii in range(M1):
                    for jj in range(M1):
                        A1[ii,jj]=F[ii,jj]


                A2 = DLFTCG(A1)
#                 A2 = A2.astype('float64')
                DET1[j] = DLFDCG(A2); DET2[j] = 1
            
            F2 = DMCRCR(LU1,LU1)
            F3 = DMCRCR(F2,LU1)
            F4 = DMCRCR(F3,LU1)
            F5 = DMCRCR(F4,LU1)
            F6 = DMCRCR(F5,LU1)
            F7 = DMCRCR(F6,LU1)

            for i in range(M1):
                for ii in range(M):
                    for jj in range(M):

                        if i == 0:
                            AG[0,ii,jj]=E[ii,jj]
                        if i == 1:
                            AG[1,ii,jj]=LU1[ii,jj]
                        if i == 2:
                            AG[2,ii,jj]=F2[ii,jj]
                        if i == 3:
                            AG[3,ii,jj]=F3[ii,jj]
                        if i == 4:
                            AG[4,ii,jj]=F4[ii,jj]
                        if i == 5:
                            AG[5,ii,jj]=F5[ii,jj]
                        if i == 6:
                            AG[6,ii,jj]=F6[ii,jj]
                        if i == 7:
                            AG[7,ii,jj]=F7[ii,jj]

            for ii in range(M):
                for jj in range(M):
                    LU2[ii,jj]=0.

            for i in range(M1):
                for ii in range(M):
                    for jj in range(M):
                        LU2[ii,jj]=LU2[ii,jj]+(DET1[i]*(10**DET2[i])*AG[i,ii,jj])/(DET10*(10**DET20))

            if n == 2:
                F2 = DMCRCR(LU3,LU2)
                continue
            else:
                for ii in range(M):
                    for jj in range(M):
                        LU3[ii,jj]=LU2[ii,jj]

        for n in range(2):
            if n == 0:
                SA=-1
            if n == 1:
                SA=1

            for ii in range(M):
                for jj in range(M):
                    LI1[ii,jj]=SA*MMT*LI[ii,jj]

            EVI = DEVLCG(LI1)

            for i in range(M):
                for j in range(M):
                    F[i,j]=EVI[i]**j # J-1 in original

            for ii in range(M):
                for jj in range(M):
                    F1[ii,jj]=F[ii,jj]

            for ii in range(M1):
                for jj in range(M1):
                    A1[ii,jj]=F[ii,jj]

            A2 = DLFTCG(A1)
            DET10 = DLFDCG(A2); DET20 = 1
            
            for j in range(M):
                for ii in range(M):
                    for jj in range(M):
                        F[ii,jj]=F1[ii,jj]

                for i in range(M):
                    F[i,j]=EX1**EVI[i]

                for ii in range(M1):
                    for jj in range(M1):
                        A1[ii,jj]=F[ii,jj]

                A2 = DLFTCG(A1)
                DET1[j] = DLFDCG(A2); DET2[j] = 1


            F2 = DMCRCR(LI1,LI1)
            F3 = DMCRCR(F2,LI1)
            F4 = DMCRCR(F3,LI1)
            F5 = DMCRCR(F4,LI1)
            F6 = DMCRCR(F5,LI1)
            F7 = DMCRCR(F6,LI1)
            
            for i in range(M1):
                for ii in range(M):
                    for jj in range(M):
                        if i == 0:
                            AG[0,ii,jj]=E[ii,jj]
                        if i == 1:
                            AG[1,ii,jj]=LI1[ii,jj]
                        if i == 2:
                            AG[2,ii,jj]=F2[ii,jj]
                        if i == 3:
                            AG[3,ii,jj]=F3[ii,jj]
                        if i == 4:
                            AG[4,ii,jj]=F4[ii,jj]
                        if i == 5:
                            AG[5,ii,jj]=F5[ii,jj]
                        if i == 6:
                            AG[6,ii,jj]=F6[ii,jj]
                        if i == 7:
                            AG[7,ii,jj]=F7[ii,jj]

            for ii in range(M):
                for jj in range(M):
                    LI2[ii,jj]=0

            for i in range(M1):
                for ii in range(M):
                    for jj in range(M):
                        LI2[ii,jj]=LI2[ii,jj]+(DET1[i]*(10**DET2[i])*AG[i,ii,jj])/(DET10*(10**DET20))
            
            if n == 2:
                continue
            else:
                for ii in range(M):
                    for jj in range(M):
                        LI3[ii,jj]=LI2[ii,jj]


        F2 = DMCRCR(LI3,LI2)
        
######################## !!!!!!! PROBLEMS START HERE WITH GG, GG1 and GG2 !!!!!!!!! #################
        
        for i in range(M20):
            for j in range(M20):
                GG[i,j]=0.
                GG1[i,j]=0.
                GG2[i,j]=0.

        for i in range(M):
            GG[i,i]=1.
            GG[i,i+M]=1.
            GG[i+M,i+2*M]=1.
            GG[i+M,i+3*M]=1.
            GG1[i,i]=1.
            GG1[i,i+M]=1.
            GG1[i+M,i+2*M]=1.
            GG1[i+M,i+3*M]=1.
        
        for i in range(M):
            for j in range(M):
                GG[i+2*M,j]=-LU[i,j]
                GG[i+2*M,j+M]=LU[i,j]
                GG[i+3*M,j+2*M]=-1*LI[i,j]
                GG[i+3*M,j+3*M]=+LI[i,j]

                GG1[i+2*M,j]=LU3[i,j]
                GG1[i+2*M,j+M]=LU2[i,j]
                GG1[i+3*M,j+2*M]=LI3[i,j]
                GG1[i+3*M,j+3*M]=LI2[i,j]

        GG2 = DLINCG(M20,GG1) # Look for M20

        
#         print("GG :", GG)
#         print("GG1 :", GG1)
#         print("GG2 :", GG2)

    
        for i in range(M):
            for j in range(M):
                HH11[i,j]=GG2[i,j]
                HH12[i,j]=GG2[i,j+M]
                HH13[i,j]=GG2[i,j+2*M]
                HH14[i,j]=GG2[i,j+3*M]
                HH21[i,j]=GG2[i+M,j]
                HH22[i,j]=GG2[i+M,j+M]
                HH23[i,j]=GG2[i+M,j+2*M]
                HH24[i,j]=GG2[i+M,j+3*M]
                HH31[i,j]=GG2[i+2*M,j]
                HH32[i,j]=GG2[i+2*M,j+M]
                HH33[i,j]=GG2[i+2*M,j+2*M]
                HH34[i,j]=GG2[i+2*M,j+3*M]
                HH41[i,j]=GG2[i+3*M,j]
                HH42[i,j]=GG2[i+3*M,j+M]
                HH43[i,j]=GG2[i+3*M,j+2*M]
                HH44[i,j]=GG2[i+3*M,j+3*M]
        
        
#         print("HH11 :", HH11);
#         print("HH12 :", HH12);
#         print("HH13 :", HH13);
#         print("HH14 :", HH14);
#         print("HH21 :", HH21);
#         print("HH22 :", HH22);
#         print("HH23 :", HH23);
#         print("HH24 :", HH24);
#         print("HH31 :", HH31);
#         print("HH32 :", HH32);
#         print("HH33 :", HH33);
#         print("HH34 :", HH34);
#         print("HH41 :", HH41);
#         print("HH42 :", HH42);
#         print("HH43 :", HH43);
#         print("HH44 :", HH44);

        
        
        F = DMCRCR(LU3,HH11)
        HH11 = DMCRCR(LU,F)
        F = DMCRCR(LU2,HH21)
        HH21 = DMCRCR(LU,F)
        F = DMCRCR(LU3,HH13)
        HH13 = DMCRCR(LU,F)
        F = DMCRCR(LU2,HH23)
        HH23 = DMCRCR(LU,F)
        F = DMCRCR(LI3,HH32)
        HH32 = DMCRCR(LI,F)
        F = DMCRCR(LI2,HH42)
        HH42 = DMCRCR(LI,F)
        F = DMCRCR(LI3,HH34)
        HH34 = DMCRCR(LI,F)
        F = DMCRCR(LI2,HH44)
        HH44 = DMCRCR(LI,F)
        
#         print("HH11: ", HH11)
#         print("HH21: ", HH21)
#         print("HH13: ", HH13)
#         print("HH23: ", HH23)
#         print("HH32: ", HH32)
#         print("HH42: ", HH42)
#         print("HH34: ", HH34)
#         print("HH44: ", HH44)
#         print("F: ", F)
        
#         for i in range(M10):
#             for j in range(M20):
#                 GG3[i,j]=0.0

        for i in range(M):
            for j in range(M):
                GG3[i,j]=-1*HH11[i,j]+HH21[i,j]
                GG3[i,j+2*M]=-1*HH13[i,j]+HH23[i,j]
                GG3[i,j+3*M]=-1*Z[i,j]
                GG3[i+M,j+M]=-1*HH32[i,j]+HH42[i,j]
                GG3[i+M,j+2*M]=-1*Y[i,j]
                GG3[i+M,j+3*M]=-1*HH34[i,j]+HH44[i,j]
        
        K1=0
        K0=0

        for j in range(M20):

            if IH[j] == 1:
                K1=K1+1
            if IH[j] == 0:
                if IH[j] == 0:
                    K0=K0+1
                if IH[j] == 1:
                    continue
                   
                for i in range(M10):
                    GG5[i,K0-1]=GG3[i,j]

            else:
                for i in range(M10):
                    GG4[i,K1-1]=-1*GG3[i,j] # Original was -GG3[i,j]
                    B10[K1-1]=B5[j]

                    
        #  !!!!!!!!!!!!!!!!!!!!        B10          !!!!!!!!!!!!!!!!!!!!!!!!


#         print("B10: ", B10)
#         print("GG4: ", GG4)
        B6 = DMUCRV(GG4,B10)     
#         print("B6: ", B6)
#         print("GG5: ", GG5)
        B7 = DLSLCG(GG5,B6)
        
#         print("B7: ", B7)  # B7 is the REAL PROBLEM. HAS TO BE CORRECTED   !!!!!!!!!!!!!!!!!!
        
#         print("GG5: ", GG5)
#         print("GG4: ", GG4)
#         print("B10: ", B10)
#         print("B6: ", B6)
#         print("B7: ", B7)
        
        K1=0

        for j in range(M20):
            if IH[j] == 0:
                K1=K1+1
            if IH[j] == 1:
                continue

            B5[j]=B7[K1-1]
#         print("B5: ", B5)  # B5 is the REAL PROBLEM. HAS TO BE CORRECTED   !!!!!!!!!!!!!!!!!!
        for i in range(M):
            UK1[i]=B5[i]
            AIK1[i]=B5[i+M]
        
#         print("Z: ", Z)
#         print("Y: ", Y)
#         print("AIK1: ", AIK1)   # AIK1 SHOLD BE CORRECTED !!!!!!!!!!!!!!!!!!!!!!!!!!
#         print("UK1: ", UK1)     # UK1 SHOLD BE CORRECTED !!!!!!!!!!!!!!!!!!!!!!!!!!
        
        AA = DMUCRV(Z,AIK1)
        BB = DMUCRV(Y,UK1)
        CC = DMCRCR(LI,LI3)
        DD = DMCRCR(LI,LI2)
        
#         print("AA:", AA)
#         print("BB:", BB)
#         print("CC:", CC)
#         print("DD:", DD)
        
        for i in range(M):
            B1[i]=UK1[i]
            B1[i+M]=AIK1[i]
            B1[i+2*M]=AA[i]
            B1[i+3*M]=BB[i]
        
        B4 = DLSLCG(GG,B1)
        
        for i in range(M):
            AA[i]=0.
            B[i]=0.
            B[i]=B4[i]

        AA = DMUCRV(LU3,B)

        for i in range(M):
            BB[i]=0.
            B[i]=0.
            B[i]=B4[i+M]

        BB = DMUCRV(LU2,B)

        for i in range(M):
            UX[i]=AA[i]+BB[i]
            if LM == MMT:
                UK1[i]=UX[i]
            UXM[i]=np.sqrt(UX[i].real**2+UX[i].imag**2)

        for i in range(M):
            AA[i]=0.
            B[i]=0.
            B[i]=B4[i+2*M]
        
        AA = DMUCRV(LI3,B)   # NEED TO CORRECT AA !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        
        for i in range(M):
            BB[i]=0.
            B[i]=0.
            B[i]=B4[i+3*M]
        
        BB = DMUCRV(LI2,B)   # NEED TO CORRECT BB !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        
        for i in range(M):
                    
            AIX[i]=AA[i]+BB[i]
            if LM == MMT:
                AIK1[i]=AIX[i]
            AIXM[i]=np.sqrt(AIX[i].real**2+AIX[i].imag**2)
            
########################### PROBLEM LIES IN    !!!!  AIX  !!!!  ###################################### 

#             print("AIX: ", AIX)
#             print("AIXM: ", AIXM)
#             print("R11: ", R11) # TOTALLY FINE ! 
            
            if i == 0 and LL == 0 and PR == 2:
                PPP1[NN,LL]=PPP1[NN,LL]+(AIXM[0]**2)/(2*R11[0])
            if i == 0 and LL > 0:
                PPP1[NN,LL]=PPP1[NN,LL]+(AIXM[0]**2)/(2*R11[0])
            if i == 1 and LL == 0 and PR == 2:
                PPP2[NN,LL]=PPP2[NN,LL]+(AIXM[1]**2)/(2*R11[1])
            if i == 1 and LL > 0:
                PPP2[NN,LL]=PPP2[NN,LL]+(AIXM[1]**2)/(2*R11[1])
            if i == 2 and LL == 0 and PR == 2:
                PPP3[NN,LL]=PPP3[NN,LL]+(AIXM[2]**2)/(2*R11[2])
            if i == 2 and LL > 0:
                PPP3[NN,LL]=PPP3[NN,LL]+(AIXM[2]**2)/(2*R11[2])
            if i == 3 and LL == 0 and PR == 2:
                PPP4[NN,LL]=PPP4[NN,LL]+(AIXM[3]**2)/(2*R11[3])
            if i == 3 and LL > 0:
                PPP4[NN,LL]=PPP4[NN,LL]+(AIXM[3]**2)/(2*R11[3])
            if i == 4 and LL == 0 and PR == 2:
                PPP5[NN,LL]=PPP5[NN,LL]+(AIXM[4]**2)/(2*R11[4])
            if i == 4 and LL > 0:
                PPP5[NN,LL]=PPP5[NN,LL]+(AIXM[4]**2)/(2*R11[4])
            if i == 5 and LL == 0 and PR == 2:
                PPP6[NN,LL]=PPP6[NN,LL]+(AIXM[5]**2)/(2*R11[5])
            if i == 5 and LL > 0:
                PPP6[NN,LL]=PPP6[NN,LL]+(AIXM[5]**2)/(2*R11[5])
            if i == 6 and LL == 0 and PR == 2:
                PPP7[NN,LL]=PPP7[NN,LL]+(AIXM[6]**2)/(2*R11[6])
            if i == 6 and LL > 0:
                PPP7[NN,LL]=PPP7[NN,LL]+(AIXM[6]**2)/(2*R11[6])
            if i == 7 and LL == 0 and PR == 2:
                PPP8[NN,LL]=PPP8[NN,LL]+(AIXM[7]**2)/(2*R11[7])
            if i == 7 and LL > 0:
                PPP8[NN,LL]=PPP8[NN,LL]+(AIXM[7]**2)/(2*R11[7])
            if LL == 0 and PR == 2:
                PPP[NN,LL]=PPP[NN,LL]+(AIXM[i]**2)/(2*R11[i])
            if LL > 0:
                PPP[NN,LL]=PPP[NN,LL]+(AIXM[i]**2)/(2*R11[i])
            if LL == 0 and PR == 1:
                PP1=PP1+(AIXM[i]**2)/(2*R11[i])
            if LL == 0 and PR == 2:
                PP2=PP2+(AIX[i]**2)/(2*R11[i])
            SM[i]=UX[i]*np.conj(AIX[i])/2
            
#             print(PPP1[NN, LL])

        # Writing UXM and AIXM values into 10th and 11th modules    

In [14]:
# !!!  470th loop stands for 1th or one prisoed of podstanst  !!!

# 11 - 13
for i in range(560+1):
    for p in range(3):
        UM[i,0,p] = funu[i,p]*np.sqrt(2) 
        FUM[i,0,p]=fu[i,p]*np.pi/180 
        UM1[i,0,p]=UM[i,0,p]*np.cos(FUM[i,0,p]) 
        UM2[i,0,p]=UM[i,0,p]*np.sin(FUM[i,0,p])     
        
# 13 - 14
for i in range(560+1):
    for p in range(3):
        for k in range(50-1):
            UM[i,k+1,p]=UM[i,k+1,p]*UM[i,0,p]/100.
            FUM[i,k+1,p]=FUM[i,k+1,p]*np.pi/180.
            UM1[i,k+1,p]=UM[i,k+1,p]*np.cos(FUM[i,k+1,p])
            UM2[i,k+1,p]=UM[i,k+1,p]*np.sin(FUM[i,k+1,p])

# 14 - 16
for i in range(560+1):
    for p in range(3):
        AIM[i,0,p]=funi[i,p]*np.sqrt(2)
        FIM[i,0,p]=fi[i,p]*np.pi/180
        AIM1[i,0,p]=AIM[i,0,p]*np.cos(FIM[i,0,p])
        AIM2[i,0,p]=AIM[i,0,p]*np.sin(FIM[i,0,p])
        
# 16 - 117
for i in range(560+1):
    for p in range(3):
        for k in range(50-1):
            AIM[i,k+1,p]=AIM[i,k+1,p]*AIM[i,0,p]/100
            FIM[i,k+1,p]=FIM[i,k+1,p]*np.pi/180
            AIM1[i,k+1,p]=AIM[i,k+1,p]*np.cos(FIM[i,k+1,p])
            AIM2[i,k+1,p]=AIM[i,k+1,p]*np.sin(FIM[i,k+1,p])

In [15]:
#  !!!!!!!!!!!!!!!!!!!!!!!!!! WATCH OUT WITH UK1 and AIK1 !!!!!!!!!!!!!!!!!!!!!

In [16]:
# 117 - 1700 - 1500

print("!!! Started !!!")

MM, MPR, MTR, DT, MT = 5, 3, 1, 2.5, 5 #         <---- INPUT FROM THE USER
M = MPR + MTR     # thus is 4
MMT = MM / MT    # is 1

if M <= 6:
    M1 = M
if M > 6:
    M1 = 6

M10 = 2 * M # is 8
M20 = 4 * M # is 16

k = LoopI(0, 49) # INNER LOOP
n = LoopI(0, 560) # OUTER LOOP
PR = LoopI(1, 2)
counter = 1

start = time.time()

while True:


    if n.the_end:
        break

    # print("START: ", n.get_current_index(), k.get_current_index())

    K = k.get_current_index(); N = n.get_current_index();
    # print(counter); counter+=1

    UK1[0] = complex(UM1[N,K,0], UM2[N,K,0])
    UK1[1] = complex(UM1[N,K,1], UM2[N,K,1])
    UK1[2] = complex(UM1[N,K,2], UM2[N,K,2])
    UK1[3] = complex(0, 0)
    AIK1[0] = complex(AIM1[N,K,0], AIM2[N,K,0])
    AIK1[1] = complex(AIM1[N,K,1], AIM2[N,K,1])
    AIK1[2] = complex(AIM1[N,K,2], AIM2[N,K,2])
    AIK1[3] = complex(0, 0)


    if not (k.get_current_index() > 0):
        if not (k.get_current_index() == 0 and PR.get_current_index() == 2):
            # print(counter); counter+=1
            UK10=(UK1[0]+UK1[1]+UK1[2])/3
            UK11=(UK1[0]+UK1[1]*AL+UK1[2]*AL**2)/3
            UK12=(UK1[0]+UK1[1]*AL**2+UK1[2]*AL)/3
            SKU2=np.sqrt(UK12.real**2+UK12.imag**2)/np.sqrt(UK11.real**2+UK11.imag**2)*100
            SKU0=np.sqrt(UK10.real**2+UK10.imag**2)/np.sqrt(UK11.real**2+UK11.imag**2)*100
            UK1[0]=UK11
            UK1[1]=UK11*AL**2
            UK1[2]=UK11*AL
            AIK10=(AIK1[0]+AIK1[1]+AIK1[2])/3
            AIK11=(AIK1[0]+AIK1[1]*AL+AIK1[2]*AL**2)/3
            AIK12=(AIK1[0]+AIK1[1]*AL**2+AIK1[2]*AL)/3
            SKI2=np.sqrt(AIK12.real**2+AIK12.imag**2)/np.sqrt(AIK11.real**2+AIK11.imag**2)*100
            SKI0=np.sqrt(AIK10.real**2+AIK10.imag**2)/np.sqrt(AIK11.real**2+AIK11.imag**2)*100
            AIK1[1]=AIK11
            AIK1[2]=AIK11*AL**2
            AIK1[3]=AIK11*AL



    # print(n.get_current_index(), k.get_current_index())
    # print(counter); counter+=1
    RASCHET(N, K)


    if k.get_current_index() == 0 and PR.get_current_index() == 1:
        PPR1[N]=PP1
    if k.get_current_index() == 0 and PR.get_current_index() == 2:
        PPR2[N]=PP2
    if k.get_current_index() == 0 and PR.get_current_index() == 1:
        PR.step_up_index()
        continue

    if PR.get_current_index() == 2:
        PR.set_current_index(1)

    if k.the_end:
        n.step_up_index()
        k.set_current_index(0)
    else:
        k.step_up_index()

print("!!! Finished !!!")
end = time.time()
print("Time took(sec): ", end - start)

!!! Started !!!
AIK1:  [-21.27431324+6.62621069j  -7.09143775+2.2087369j
  -0.        +0.j          -0.        +0.j        ]
AIK1:  [-21.27431324 +6.62621069j  15.91548683+14.7379024j
   4.84774643-20.83194918j   0.         +0.j        ]
AIK1:  [0.00401473-0.0696281j  0.04524517-0.00284658j 0.04315718+0.11012414j
 0.        +0.j        ]
AIK1:  [ 0.31250521+0.05622856j -0.32950551+0.01150659j -0.11111983-0.07079114j
  0.        +0.j        ]
AIK1:  [-0.05126974-0.02500592j -0.03283007+0.04585656j -0.00724431+0.00494174j
  0.        +0.j        ]
AIK1:  [-0.70556767+0.36105722j -0.85638566+0.54347887j -0.99872464+0.38537576j
  0.        +0.j        ]
AIK1:  [ 0.09009449+0.16253477j -0.03711514+0.12293126j  0.07715037+0.19788046j
  0.        +0.j        ]
AIK1:  [0.21888989-0.05865137j 0.08570626-0.15461819j 0.20874567-0.09381402j
 0.        +0.j        ]
AIK1:  [-0.02056541-0.01845229j -0.02244157-0.02013567j -0.02267145+0.02084747j
  0.        +0.j        ]
AIK1:  [-0.09461524+0.066496

AIK1:  [-0.03400806+0.01151109j  0.00378414-0.01294074j -0.01963469+0.00734111j
  0.        +0.j        ]
AIK1:  [-0.00165753+0.01753482j -0.00263528-0.00030025j -0.00852216+0.0248912j
  0.        +0.j        ]
AIK1:  [ 0.0040039 -0.04158206j  0.03066874+0.00182206j -0.02817918+0.03075218j
  0.        +0.j        ]
AIK1:  [-0.02265372-0.01174257j -0.0162538 +0.01896364j -0.01067885-0.03876933j
  0.        +0.j        ]
AIK1:  [-0.08831881-0.00865973j -0.0360876 -0.08143631j -0.03450695-0.0277443j
  0.        +0.j        ]
AIK1:  [-0.0283006 +0.0046344j  -0.00990827-0.03302599j -0.02634656+0.01595605j
  0.        +0.j        ]
AIK1:  [ 0.0131182 -0.02701587j -0.02424509-0.06808812j -0.04269963-0.01707863j
  0.        +0.j        ]
AIK1:  [-0.03017594+0.03081466j  0.00374831-0.00026211j  0.0256612 -0.00823891j
  0.        +0.j        ]
AIK1:  [ 0.01201682-0.03788181j -0.00997865-0.02741612j -0.00946124-0.00177062j
  0.        +0.j        ]
AIK1:  [-0.00144369+0.02117675j  0.02519487+0.00

AIK1:  [-0.02681941-0.02654002j -0.00199903-0.00455392j -0.05411644-0.03213199j
  0.        +0.j        ]
AIK1:  [-0.12004999+0.03970274j -0.17112608-0.02496459j -0.03601508-0.08525964j
  0.        +0.j        ]
AIK1:  [ 0.01538043+0.0074021j  -0.02680438-0.00545341j  0.01582152+0.02107229j
  0.        +0.j        ]
AIK1:  [0.02176556-0.01946066j 0.10398449+0.0548248j  0.02566337+0.02452445j
 0.        +0.j        ]
AIK1:  [-0.00313925-0.02178492j -0.00316358-0.00122072j -0.02803602-0.02923569j
  0.        +0.j        ]
AIK1:  [-0.00201796+0.06083077j -0.00952939+0.00933187j  0.01746853-0.01024873j
  0.        +0.j        ]
AIK1:  [-0.01218795+0.01913125j -0.0188655 +0.04009122j -0.006296  -0.00049551j
  0.        +0.j        ]
AIK1:  [ 0.01328076-0.00961367j -0.00552363-0.02215405j -0.01066312-0.06732429j
  0.        +0.j        ]
AIK1:  [-0.01835312-0.01173731j  0.0235639 -0.02654066j -0.007473  -0.01525428j
  0.        +0.j        ]
AIK1:  [-0.00699093+0.00991027j -0.04975246+0.0105

AIK1:  [ 0.04439275-0.02223023j -0.00520433-0.01429877j  0.02238745+0.0315022j
  0.        +0.j        ]
AIK1:  [-0.80447073-0.56749168j -1.03682847-0.54665764j -0.89395832-0.71108655j
  0.        +0.j        ]
AIK1:  [ 0.02400234+0.09281014j -0.00739962-0.03205126j -0.01442279-0.04310518j
  0.        +0.j        ]
AIK1:  [0.195819  +0.13916132j 0.20772423+0.10492906j 0.20451482+0.11476837j
 0.        +0.j        ]
AIK1:  [-0.03212771+0.02465246j -0.02393104+0.01115923j  0.02987769-0.00350982j
  0.        +0.j        ]
AIK1:  [-0.00190224-0.00918555j -0.0855303 +0.01492743j  0.02461493-0.00563757j
  0.        +0.j        ]
AIK1:  [-0.01938846-0.03371756j -0.03585805+0.02284409j  0.02348438+0.00330051j
  0.        +0.j        ]
AIK1:  [-0.28276185+0.55256511j -0.20525781+0.54319904j -0.19312097+0.52207383j
  0.        +0.j        ]
AIK1:  [ 0.009811  +0.00493442j -0.01399557-0.00396035j -0.00124398-0.01421871j
  0.        +0.j        ]
AIK1:  [-0.15094674+0.41472276j -0.09522716+0.45582

AIK1:  [-0.03352115+0.00376j     0.01853822-0.00313551j  0.01189537-0.0001661j
  0.        +0.j        ]
AIK1:  [-0.00972153-0.07917554j -0.03025262-0.00500829j  0.00559386+0.00871333j
  0.        +0.j        ]
AIK1:  [-0.00465591+0.00110919j  0.00614265+0.01634224j  0.0174886 -0.02011833j
  0.        +0.j        ]
AIK1:  [-0.02583856-0.02732351j  0.00192953-0.00152933j -0.03889666+0.02545326j
  0.        +0.j        ]
AIK1:  [-0.00308766+0.01588463j -0.06545527+0.01025002j -0.00495138-0.01413891j
  0.        +0.j        ]
AIK1:  [ 0.01565749+0.02595572j  0.02636317-0.00379899j -0.01712552-0.01488699j
  0.        +0.j        ]
AIK1:  [ 1.69106697e-02-0.03127563j -4.39946117e-18-0.02394955j
 -7.49883937e-03-0.00776528j  0.00000000e+00+0.j        ]
AIK1:  [ 0.04827988+0.00507442j -0.03425301-0.01119561j  0.01019244+0.00569636j
  0.        +0.j        ]
AIK1:  [ 0.02860356+0.0025528j  -0.02034728+0.02585678j -0.0475992 +0.00788j
  0.        +0.j        ]
AIK1:  [-0.04551449+0.04729656j -0

...

AIK1:  [0.02930783-0.01037845j 0.02933046+0.01572685j 0.0106226 -0.0186234j
 0.        +0.j        ]
AIK1:  [ 0.0229674 +0.03036835j -0.00317268+0.00835214j  0.02256516-0.00446802j
  0.        +0.j        ]
AIK1:  [0.01452963-0.0209054j  0.02329808-0.023136j   0.03550794+0.00037185j
 0.        +0.j        ]
AIK1:  [-0.0096111 +0.03637656j  0.01353548-0.02492926j -0.00050127+0.00735291j
  0.        +0.j        ]
AIK1:  [-0.01281036+0.01202973j -0.04016969+0.00168361j  0.00163949-0.01374922j
  0.        +0.j        ]
AIK1:  [-0.03480802-0.01239458j  0.01552349-0.00324303j  0.06544256-0.04233669j
  0.        +0.j        ]
AIK1:  [-0.00089763+8.01109581e-05j -0.01559591-1.66079543e-02j
  0.03437227+1.20030611e-03j  0.        +0.00000000e+00j]
AIK1:  [0.0435815 +0.0786231j  0.02766112-0.01769003j 0.01744053-0.02208354j
 0.        +0.j        ]
AIK1:  [-0.00803257+0.00618591j -0.00011305-0.06477456j -0.04582942-0.00758701j
  0.        +0.j        ]
AIK1:  [ 0.00716951-0.00954889j -0.00312129

AIK1:  [-0.00992808-0.01843845j  0.01427175+0.01245004j  0.0189434 -0.0119293j
  0.        +0.j        ]
AIK1:  [ 0.05989261-0.04224959j -0.01070638-0.10919208j  0.00243532-0.0302682j
  0.        +0.j        ]
AIK1:  [-0.0309237 +0.01143902j  0.03744587+0.01999423j -0.01105465+0.02197972j
  0.        +0.j        ]
AIK1:  [ 0.04102646+0.0094717j  -0.00348836+0.00222233j -0.00841216+0.03672944j
  0.        +0.j        ]
AIK1:  [-0.0290107 +0.00948216j -0.0244738 +0.00973931j  0.05423707+0.01741362j
  0.        +0.j        ]
AIK1:  [-0.04546753-0.01278035j -0.02943673+0.0587838j  -0.03038447+0.01913412j
  0.        +0.j        ]
AIK1:  [ 0.00695823+0.02116358j -0.01308981+0.02872296j  0.00091621+0.02281153j
  0.        +0.j        ]
AIK1:  [0.00805134-0.00261604j 0.02427937+0.04724254j 0.00888047+0.01217817j
 0.        +0.j        ]
AIK1:  [ 0.        +0.j         -0.00523983-0.01955532j -0.01658547-0.02107638j
  0.        +0.j        ]
AIK1:  [ 0.00408229+0.01178894j  0.02653449-0.023229

AIK1:  [0.19187358-0.02764941j 0.0712976 +0.06885129j 0.23915891+0.17312218j
 0.        +0.j        ]
AIK1:  [ 0.0646122 -0.03263797j -0.03072686+0.14455851j -0.01326527+0.02393118j
  0.        +0.j        ]
AIK1:  [-0.07420442-0.01919057j -0.07864375-0.029717j    0.09384577+0.07902548j
  0.        +0.j        ]
AIK1:  [-0.02925451+0.03306621j -0.05678234+0.00919673j  0.04831522-0.10601808j
  0.        +0.j        ]
AIK1:  [-0.3697726 +0.45177384j -0.18639047+0.42460935j -0.23604669+0.56428777j
  0.        +0.j        ]
AIK1:  [ 0.03461375-0.01018738j  0.01605254+0.00691332j -0.02554422-0.02563354j
  0.        +0.j        ]
AIK1:  [-0.27038314+0.38329253j -0.10483021+0.41734656j -0.19062474+0.27944541j
  0.        +0.j        ]
AIK1:  [0.0691659 +0.0065381j  0.0567912 +0.04517376j 0.01068294+0.03388196j
 0.        +0.j        ]
AIK1:  [-0.06167983-0.07020867j  0.01465808-0.06200803j  0.03343896+0.01748146j
  0.        +0.j        ]
AIK1:  [-0.0166996 +0.03349421j  0.01151578+0.02538639

AIK1:  [-0.01692578-0.00085742j -0.0046609 -0.02466714j  0.03017548+0.0145874j
  0.        +0.j        ]
AIK1:  [ 0.00832019+0.03492451j -0.00192918+0.03562136j  0.00095764-0.03657076j
  0.        +0.j        ]
AIK1:  [ 0.00488605+0.0136462j   0.01972771-0.04472872j -0.00546976+0.04876401j
  0.        +0.j        ]
AIK1:  [0.04577514-0.00594515j 0.03015927+0.01175861j 0.029625  +0.01703513j
 0.        +0.j        ]
AIK1:  [ 0.04783048-0.04352239j  0.00473337-0.00522018j -0.01344806+0.01223679j
  0.        +0.j        ]
AIK1:  [ 0.02257911-0.02090849j -0.0209684 -0.02886052j -0.02056284-0.00535622j
  0.        +0.j        ]
AIK1:  [-0.07174082+0.00300683j -0.00360355-0.01836745j -0.02277894+0.00039761j
  0.        +0.j        ]
AIK1:  [-0.02598283+0.00236462j  0.02318601-0.05307068j -0.03222612+0.01684741j
  0.        +0.j        ]
AIK1:  [ 0.00200887-0.02802524j -0.03883365+0.0558743j  -0.10569979+0.04750337j
  0.        +0.j        ]
AIK1:  [ 0.01530059+0.05407115j -0.00750992+0.01617

AIK1:  [-0.13397746+0.11362115j -0.06841719+0.15152729j -0.04277155+0.12636311j
  0.        +0.j        ]
AIK1:  [ 0.04627468-0.02450118j -0.01248655+0.00182159j -0.01683848-0.01140051j
  0.        +0.j        ]
AIK1:  [0.06029704+0.06557254j 0.07850223+0.01089308j 0.11126313+0.02466644j
 0.        +0.j        ]
AIK1:  [ 0.00998219+0.02284833j -0.00557682-0.00981697j  0.0249615 -0.00849759j
  0.        +0.j        ]
AIK1:  [ 0.04087535-0.0070604j   0.01991373+0.03477133j -0.00181272+0.03458881j
  0.        +0.j        ]
AIK1:  [-0.02174572+0.00079738j  0.00266684+0.00407536j -0.00629316-0.01055687j
  0.        +0.j        ]
AIK1:  [0.01987146-0.02617969j 0.02758568-0.02186414j 0.0377297 -0.03555467j
 0.        +0.j        ]
AIK1:  [-0.04190411-0.02488084j -0.0103365 +0.01465293j -0.02868171+0.00688587j
  0.        +0.j        ]
AIK1:  [-0.02494142-0.01733474j -0.03125845+0.02814523j -0.06724013+0.04655928j
  0.        +0.j        ]
AIK1:  [ 0.01194871-0.01597217j  0.00207373+0.00560603

In [24]:
# 1500 - 1501

for n in range(560):
    PRP = 0
    for k in range(50): # 1060'th loop
        PRP=PRP+PPP[n,k]
    RPR=0
    for k in range(1, 50): # 1061'st loop
        RPR=RPR+PPP[n,k]
    
    SS1=(PPP[n,0]/PPR1[n]-1)*100
    SS2=(RPR/PPR1[n])*100
    SS0=PPP[n,0]-PPR1[n]
    
    # WRITING INTO "PPP" and "PPPx" files corresponding PPP and PPPx values
#     print(PPP1[n, k])
    
# 1501 - 1052
WD0=0.
for k in range(50):
    WD[0,k]=0
    for i in range(560):
        WD0=WD0+PPP[i,k]*DT/60000
        WD[0,k]=WD[0,k]+PPP[i,k]*DT/60000

# 1052 - 1057
WD1=WD0-WD[0,0]
for k in range(50):
    WD[1,k]=WD[0,k]/WD0*100
WD4=0
for k in range(13, 50):
    WD4=WD4+WD[1,k]
WD10=0
for i in range(560):
    WD10=WD10+PPR1[i]*DT/60000
 
# 1057 -1054
for i in range(560):
    PD[0,i] = 0
    PD[1,i] = 0
    for k in range(50):
        PD[0,i]=PD[0,i]+PPP[i,k]
    PD[1,i]=PD[0,i]-PPP[i,0]
    PD[2,i]=PD[1,i]/PD[0,i]*100

# WRITING CALCULATION RESULTS INTO 8th UNIT (Результаты расчета)

C:\Users\73B5~1\AppData\Local\Temp/ipykernel_5356/3449162367.py:11: RuntimeWarning: invalid value encountered in double_scalars
  SS1=(PPP[n,0]/PPR1[n]-1)*100
C:\Users\73B5~1\AppData\Local\Temp/ipykernel_5356/3449162367.py:12: RuntimeWarning: divide by zero encountered in double_scalars
  SS2=(RPR/PPR1[n])*100


In [25]:
for item in PPP1[:560,1]:
    print(item)

22.106741959110302
25.406970445214363
40.645578877979055
58.319950230444704
49.94029249518224
3.328342452227529
6.597236201969279
20.16451560365281
11.331657967268496
1.6591915342868107
0.7747764004628015
5.763518189808966
7.558269172332675
47.026625131026556
107.55107496401598
288.770982817107
29.509829640243346
5.085814323093572
20.852342427233314
55.61461599933428
76.47070475065166
36.971295842828425
98.36023967147496
1.602110592912706
43.873152175144824
55.97513296077783
129.20742932257406
38.80305215765409
29.900548663957913
0.5962190588745346
68.03305688978323
23.179801841345473
51.38261604465135
56.27796796006017
121.73340393776176
0.8937259316972993
98.6262261000154
11.4941613918525
49.408363504211415
9.735051508411141
57.07575463728492
25.25367055076874
52.54832350637757
1.498609440713907
35.651530773295185
129.7086879769439
42.99261257384009
54.20969847494095
84.21237813209356
7.580199304918558
2.189245566492361
109.87838055441563
107.4631157772896
52.78137889648646
15.769299

In [26]:
int(WD0) # HAVE TO BE AROUND ~ 16.79

52225929628266672